# Urban Computing || Greg Dobler || July 10, 2017
-----------------------------------------------------------

## Structure of this module
Problems/topics/readings/videos will be suggested <br>
One (potentially optional) homework will be given that will consist of submitting code <br>
Useful reading: <i>Programming Computer Vision with Python</i>, Jan Erik Solem

<br>

## Delving further into scientific python
NumPy -- arrays (2D and 3D), vectorized operations, basic functionality <br>
SciPy -- functions, transformations, analysis <br>
Matplotlib -- plotting, viewing, visualizations (static and interactive)

<br>

## Image processing and Computer Vision
What are images from a data perspective? <br>
How are images loaded displayed and handled? <br>
What are some basic image processing taks? <br>
What can we learn from images? <br><br>
<b>Video is not particularly <i>special</i>, it's just a series of images to be analyzed together</b>

----------------------

## 1D
Let's look at a "time series".  First some imports that will be useful:

In [1]:
import os
import sys
import time
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

Now we load in the one-dimensional data (which has been saved as a numpy array) and define a plotting function to view the data in a couple of different ways.

In [2]:
data = np.load('output/ml_flat.npy')

We'll be using matplotlib to visualize interactively (**not** in inline mode), so let's switching mpl's interactive mode to "on":

In [3]:
plt.ion()

In [4]:
fig0 = plt.figure(0)
def showit(fac, sep=100.):
    fig0.clf()
    ax0 = fig0.add_subplot(1, 1, 1)
    lin0 = ax0.plot(range(data.size // fac), 
                    data.reshape(data.size // fac, fac) + 
                    sep * np.arange(fac), lw=0.05, color='k')
    fig0.canvas.draw()

In [5]:
showit(1)

This one-dimensional "time series" can be sliced and plotted stacked on top of itself.  For example, cut it in half and plot the two curves:

In [6]:
showit(2, 200)

If we progressively cut further, stacking each time...

In [7]:
showit(4, 200)

In [8]:
showit(20, 200)

In [9]:
showit(954, 10)

In [10]:
fig1, ax1 = plt.subplots(num=1)
im1 = ax1.imshow(data.reshape(data.size // 954, 954).T[::-1, :], 
                 cmap="gist_gray")
ax1.grid(0)
fig1.canvas.draw()

---

## 1D vs 2D arrays

Grayscale images are 2D <b>arrays</b> but in many cases can be thought of (and treated as) 1D arrays.<br>
<br>
Many data analysis tasks you've already seen can be handled exactly the same:<br>
. correlations<br>
. means and variances<br>
. multivariate regressions<br>
<br>
others will be different:<br>
. filters<br>
. derivatives<br>
. rotations<br>
<br>
But since this is a numpy array, things are easily <b><i>vectorized</i></b> and fast.  For example, let's look at derivatives.  First the 1D case:

In [11]:
xx = np.linspace(-20, 20, 1000)
yy = np.cos(xx)

In [12]:
fig3, ax3 = plt.subplots(num=3)
lin3a, = ax3.plot(xx, yy, "darkred")
lin3b, = ax3.plot(xx[1:], (yy[1:] - yy[:-1]) / (xx[1] - xx[0]),
                  "dodgerblue")
fig3.canvas.draw()

Which can be used to find edges,

In [13]:
xx  = np.linspace(-20, 20, 1000)
yy  = 1.0 * (xx > 4.3)
off = -0.1

In [14]:
lin3a.set_data(xx, yy)
lin3b.set_data(xx[1:], (yy[1:] - yy[:-1]) / (xx[1] - xx[0])
              + off)
lin3b.set_linewidth(0.5)
lin3b.set_visible(False)
fig3.canvas.draw()

In [15]:
lin3b.set_visible(True)
fig3.canvas.draw()

The same thing is true in 2D, but now we can do all rows at once,

In [16]:
ncol = 954
img  = data.reshape(data.size // ncol, ncol).T[::-1, :]
print("image shape is {0}".format(img.shape))

image shape is (954, 640)


In [17]:
plt.rcParams["image.cmap"] = "gist_gray"
fig4, ax4 = plt.subplots(1, 2, num=4)
[i.grid('off') for i in ax4]
im4a = ax4[0].imshow(img)
im4b = ax4[1].imshow(img[:, 5:] - img[:, :-5])
fig4.canvas.draw()

... or we can do all columns at once

In [18]:
im4b.set_data(img[5:, :] - img[:-5, :])
fig4.canvas.draw()

For completeness, let's do rows and columns (not simultaneously!) take the absolute value and add them together.  This is the simplest version of an edge detector,

In [19]:
edges_simp = np.abs(img[5:, 5:] - img[:-5, 5:]) + \
    np.abs(img[5:, 5:] - img[5:, :-5])

In [20]:
im4b.set_data(edges_simp)
fig4.canvas.draw()

which is often combined with thresholding

In [22]:
edges_simp.max()

238.66666666666666

In [21]:
im4b.set_data(edges_simp > 60)
im4b.set_clim(0, 1)
fig4.canvas.draw()

You can see it's not doing great, it's missing some edges and there's a lot of noise.  We'll fix this later...

---

In [22]:
plt.close("all")